In [3]:
import os
import pyrosm
import snail

In [3]:
osm = pyrosm.OSM(os.path.join('data','osm','laos-latest-highway.osm.pbf'))

In [4]:
network = osm.get_network(network_type="driving")

In [6]:
network.columns

Index(['access', 'area', 'bicycle', 'bicycle_road', 'bridge', 'cycleway',
       'est_width', 'foot', 'highway', 'int_ref', 'junction', 'lanes', 'lit',
       'maxspeed', 'motorcar', 'motorroad', 'motor_vehicle', 'name', 'oneway',
       'psv', 'ref', 'service', 'sidewalk', 'smoothness', 'surface',
       'tracktype', 'tunnel', 'width', 'id', 'timestamp', 'version', 'tags',
       'osm_type', 'geometry', 'length'],
      dtype='object')

In [7]:
nodes, edges = osm.get_network(nodes=True, network_type="driving")

In [26]:
edges.columns

Index(['access', 'area', 'bicycle', 'bicycle_road', 'bridge', 'cycleway',
       'est_width', 'foot', 'highway', 'int_ref', 'junction', 'lanes', 'lit',
       'maxspeed', 'motorcar', 'motorroad', 'motor_vehicle', 'name', 'oneway',
       'psv', 'ref', 'service', 'sidewalk', 'smoothness', 'surface',
       'tracktype', 'tunnel', 'width', 'id', 'timestamp', 'version', 'tags',
       'osm_type', 'geometry', 'u', 'v', 'length'],
      dtype='object')

In [46]:
edges['area'].value_counts()

yes    31
Name: area, dtype: int64

In [47]:
edges.oneway.value_counts()

yes    11752
no      2310
-1         4
Name: oneway, dtype: int64

In [28]:
core = (
    'motorway_link',
    'motorway',
    'trunk_link',
    'trunk',
    'primary_link',
    'primary',
    'secondary_link',
    'secondary',
    'tertiary_link',
    'tertiary',
)
core_edges = edges[edges.highway.isin(core)]

In [30]:
len(core_edges), len(edges)

(379127, 1468629)

In [49]:
select_columns = [
    'bridge', 'highway', 'lanes', 'maxspeed', 'oneway',
    'smoothness', 'surface', 'tracktype', 'tunnel', 'width', 
    'id', 'name', 'osm_type', 'geometry', 'u', 'v', 'length'
]

In [50]:
core_edges[select_columns].to_file(os.path.join('data', 'osm', 'laos-roads-core.gpkg'), driver='GPKG')

In [51]:
# Write direct from pyrosm driving
edges.to_file(os.path.join('data', 'osm', 'laos-roads.gpkg'), driver='GPKG', layer='edges')
nodes.to_file(os.path.join('data', 'osm', 'laos-roads.gpkg'), driver='GPKG', layer='nodes')

In [4]:
import snail
import geopandas

In [5]:
core_edges = geopandas.read_file(os.path.join('data', 'osm', 'laos-roads-core.gpkg'))

/ouce-home/staff/mert2014/.conda/envs/snail/lib/python3.8/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [6]:
from snail.intersections import split
from snail.intersections import get_cell_indices

In [7]:
import rasterio

In [9]:
raster_data = rasterio.open(os.path.join('..', 'aqueduct', 'inuncoast_historical_nosub_hist_rp0050_0.tif'))

In [11]:
# from tqdm.notebook import tqdm

In [19]:
core_splits = []
for edge in core_edges.itertuples():
    splits = split(
        edge.geometry,
        raster_data.width,
        raster_data.height,
        list(raster_data.transform),
    )
    for s in splits:
        core_splits.append({
            'id': edge.id,
            'geometry': s
        })
core_splits = geopandas.GeoDataFrame(core_splits)

In [27]:
core_edges[['id','geometry']].head(50).tail()

,id,geometry
45,10874152,"LINESTRING (102.62496 17.95004, 102.62505 17.9..."
46,10874152,"LINESTRING (102.62505 17.95043, 102.62509 17.9..."
47,10874152,"LINESTRING (102.62509 17.95061, 102.62521 17.9..."
48,10874213,"LINESTRING (102.63006 17.95827, 102.63009 17.9..."
49,10874213,"LINESTRING (102.63009 17.95826, 102.63025 17.9..."


In [29]:
core_splits['cell_index'] = core_splits.geometry.apply(
    lambda geom: get_cell_indices(geom, raster_data.width, raster_data.height, list(raster_data.transform)))

In [30]:
core_splits.head()

,id,geometry,cell_index
0,10639496,"LINESTRING (102.61857 17.95361, 102.61848 17.9...","(33914, 8645)"
1,10639496,"LINESTRING (102.61848 17.95358, 102.61739 17.9...","(33914, 8645)"
2,10640794,"LINESTRING (102.61757 17.97053, 102.61785 17.9...","(33914, 8643)"
3,10641308,"LINESTRING (102.60515 17.96999, 102.60517 17.9...","(33912, 8643)"
4,10641308,"LINESTRING (102.60517 17.97003, 102.60519 17.9...","(33912, 8643)"


In [34]:
band = raster_data.read(1)

In [46]:
core_splits['inuncoast_historical_nosub_hist_rp0050_0'] = core_splits.cell_index.apply(lambda i: band[i[1], i[0]])

In [38]:
raster_data.width, raster_data.height

(43200, 21600)

In [52]:
core_splits.cell_index.value_counts()

(34055, 8300)    249
(34165, 8373)    216
(34450, 8842)    215
(34158, 8371)    212
(34161, 8373)    212
                ... 
(33903, 8536)      1
(33900, 8536)      1
(33967, 8451)      1
(34323, 8820)      1
(33910, 8635)      1
Name: cell_index, Length: 22488, dtype: int64

In [53]:
core_splits

,id,geometry,cell_index,inuncoast_historical_nosub_hist_rp0050_0
0,10639496,"LINESTRING (102.61857 17.95361, 102.61848 17.9...","(33914, 8645)",0.0
1,10639496,"LINESTRING (102.61848 17.95358, 102.61739 17.9...","(33914, 8645)",0.0
2,10640794,"LINESTRING (102.61757 17.97053, 102.61785 17.9...","(33914, 8643)",0.0
3,10641308,"LINESTRING (102.60515 17.96999, 102.60517 17.9...","(33912, 8643)",0.0
4,10641308,"LINESTRING (102.60517 17.97003, 102.60519 17.9...","(33912, 8643)",0.0
...,...,...,...,...
394535,976227965,"LINESTRING (103.36989 19.32709, 103.36981 19.3...","(34004, 8480)",0.0
394536,976227966,"LINESTRING (103.36981 19.32657, 103.36976 19.3...","(34004, 8480)",0.0
394537,978320012,"LINESTRING (100.75895 19.57750, 100.75913 19.5...","(33691, 8450)",0.0
394538,978320012,"LINESTRING (100.75913 19.57750, 100.75930 19.5...","(33691, 8450)",0.0


In [54]:
from glob import glob

In [56]:
fnames = glob('../aqueduct/*.tif')
fnames[0]

'../aqueduct/inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00050.tif'

In [62]:
for fname in fnames:
    colname = os.path.basename(fname[:-4])
    print(colname)
    with rasterio.open(fname) as dataset:
        band = dataset.read(1)
        core_splits[colname] = core_splits.cell_index.apply(lambda i: band[i[1], i[0]])

inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00050
inuncoast_rcp4p5_nosub_2030_rp0050_0
inuncoast_rcp4p5_wtsub_2050_rp0500_0_perc_50
inuncoast_rcp8p5_wtsub_2080_rp0010_0_perc_50
inuncoast_rcp4p5_wtsub_2050_rp0250_0
inuncoast_rcp8p5_nosub_2050_rp1000_0
inuncoast_rcp4p5_nosub_2030_rp0500_0_perc_05
inunriver_rcp4p5_00000NorESM1-M_2030_rp00005
inunriver_rcp4p5_0000GFDL-ESM2M_2030_rp00250
inuncoast_historical_nosub_hist_rp0010_0
inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00100
inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp01000
inunriver_rcp8p5_0000HadGEM2-ES_2030_rp00500
inuncoast_rcp8p5_nosub_2050_rp0001_5_perc_50
inuncoast_rcp8p5_wtsub_2080_rp0100_0_perc_05
inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp00500
inuncoast_rcp8p5_wtsub_2080_rp0002_0
inuncoast_rcp8p5_wtsub_2050_rp0002_0_perc_05
inuncoast_rcp8p5_wtsub_2030_rp0001_5_perc_05
inunriver_rcp4p5_00000NorESM1-M_2030_rp00002
inuncoast_rcp8p5_nosub_2030_rp0002_0_perc_50
inuncoast_rcp8p5_wtsub_2030_rp0001_5
inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00025
inuncoast

KeyboardInterrupt: 

In [65]:
core_splits.drop(columns='geometry').to_csv('core_splits.csv.gz')

In [66]:
core_splits[['id','geometry']].to_file('core_splits.gpkg', driver='GPKG')